In [ ]:
!pip install -q -U google-generativeai

In [20]:
import google.generativeai as genai
from google.colab import userdata
from google.generativeai.types import Tool, FunctionDeclaration
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from getpass import getpass
import os

# Retrieve the API key from Colab Secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
username = input("Enter your name: ")
os.environ['SENDER_EMAIL'] = input("Enter your email: ")
os.environ['SENDER_PASSWORD'] = getpass("Enter your email AppPassword, generate from here myaccount.google.com/apppasswords")
def send_mail(to: str, subject: str, body: str,name: str, SENDER_EMAIL: str, SENDER_PASSWORD: str):
  """
    Sends an email to anyone which is required.
    Parameters:
      to: str, subject: str, body: str
    Returns:
      boolean: True if email sent successfully
  """
  RECEIVER_EMAIL = to # Can be a single email or a list
  SUBJECT = subject
  BODY = body

  # --- Create the Email Message ---
  msg = MIMEMultipart()
  msg['From'] = SENDER_EMAIL
  msg['To'] = RECEIVER_EMAIL
  msg['Subject'] = SUBJECT
  msg.attach(MIMEText(BODY, 'plain')) # Use 'plain' for plain text, 'html' for HTML content

  # --- Send the Email ---
  try:
      print(f"Attempting to send email from {SENDER_EMAIL} to {RECEIVER_EMAIL}...")

      # Connect to Gmail's SMTP server
      server = smtplib.SMTP('smtp.gmail.com', 587)
      server.starttls() # Enable TLS encryption for secure communication

      # Log in to your Gmail account using the App Password
      server.login(SENDER_EMAIL, SENDER_PASSWORD)

      # Send the email
      text = msg.as_string()
      server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, text)

      print("Email sent successfully!")
      return True

  except Exception as e:
      print(f"An error occurred: {e}")
      return False

  finally:
      if 'server' in locals() and server:
          server.quit()          print("SMTP server connection closed.")


email_tool_declaration = FunctionDeclaration(
    name="send_mail",
    description=f"Sends email to anyone who user prompts to, write {username} if sender name needed",
    parameters={
        "type": "OBJECT",
        "properties": {
            "to": {"type": "STRING", "description": "The receiver of the email e.g. 'david@gmail.com'"},
            "subject": {"type": "STRING", "description": "The subject line of the email e.g. 'Meeting Reminder'"},
            "body": {"type":"STRING","description":"The body of the email"}
        },
        "required": ['to'],
    },
)
available_tools = [
    Tool(function_declarations=[email_tool_declaration])
]

model = genai.GenerativeModel('gemini-2.0-flash', system_instruction='you are an agent who can send emails', tools=available_tools)

response = model.generate_content(input("Prompt: "))

if 'function_call' in response.candidates[0].content.parts[0]:
  func_call = response.candidates[0].content.parts[0].function_call
  func_args = func_call.args
  if func_call.name == "send_mail":
    SENDER_EMAIL = os.environ.get('SENDER_EMAIL')
    SENDER_PASSWORD = os.environ.get('SENDER_PASSWORD')
    send_mail(func_args['to'], func_args['subject'], func_args['body'],username, SENDER_EMAIL, SENDER_PASSWORD)
else:
  print(response.candidates[0].content.parts[0].text)